# Import

In [3]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import numpy as np
import random
import os
import matplotlib.pyplot as plt

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything(42)

# Load Data

In [ ]:
# csv to parquet (메모리에ㅐ)
# def csv_to_parquet(csv_path, save_name):
#     df = pd.read_csv(csv_path)
#     df.to_parquet(f'./{save_name}.parquet')
#     del df
#     gc.collect()
#     print(save_name, 'Done.')

# csv_to_parquet('./dts/train.csv','train')
# csv_to_parquet('./dts/test.csv','test')

In [5]:
train = pd.read_parquet('./train.parquet').drop("ID", axis=1)
test = pd.read_parquet('./test.parquet').drop("ID", axis=1)
sample_submission = pd.read_csv('./dts/sample_submission.csv')

In [30]:
# na 값 확인
train.isna().sum()

Month                            0
Day_of_Month                     0
Estimated_Departure_Time    109019
Estimated_Arrival_Time      109040
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                109015
Destination_Airport              0
Destination_Airport_ID           0
Destination_State           109079
Distance                         0
Airline                     108920
Carrier_Code(IATA)          108990
Carrier_ID(DOT)             108997
Tail_Number                      0
Delay                       744999
dtype: int64

# Preprocessing

In [45]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 18 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   Month                     1000000 non-null  int64  
 1   Day_of_Month              1000000 non-null  int64  
 2   Estimated_Departure_Time  890981 non-null   float64
 3   Estimated_Arrival_Time    890960 non-null   float64
 4   Cancelled                 1000000 non-null  int64  
 5   Diverted                  1000000 non-null  int64  
 6   Origin_Airport            1000000 non-null  object 
 7   Origin_Airport_ID         1000000 non-null  int64  
 8   Origin_State              890985 non-null   object 
 9   Destination_Airport       1000000 non-null  object 
 10  Destination_Airport_ID    1000000 non-null  int64  
 11  Destination_State         890921 non-null   object 
 12  Distance                  1000000 non-null  float64
 13  Airline                   89

In [46]:
train['Carrier_ID(DOT)'].unique()

array([19393., 20304., 19805.,    nan, 20452., 20366., 19790., 20046.,
       19687., 20409., 20416., 19930., 19977., 20378., 20436., 20237.,
       20397., 20363., 20427., 20398., 19690., 20368., 21171., 20225.,
       21167., 20500., 20445., 20263., 20253.])

In [44]:
train[['Origin_State','Destination_State','Airline','Carrier_Code(IATA)','Carrier_ID(DOT)']].fillna('Null_Value')

,Origin_State,Destination_State,Airline,Carrier_Code(IATA),Carrier_ID(DOT)
0,Oklahoma,Texas,Southwest Airlines Co.,WN,19393.0
1,Illinois,Utah,SkyWest Airlines Inc.,UA,20304.0
2,North Carolina,New York,American Airlines Inc.,AA,19805.0
3,California,New Jersey,United Air Lines Inc.,UA,Null_Value
4,California,California,SkyWest Airlines Inc.,UA,20304.0
...,...,...,...,...,...
999995,Null_Value,Null_Value,United Air Lines Inc.,UA,19977.0
999996,Null_Value,Minnesota,SkyWest Airlines Inc.,DL,Null_Value
999997,Null_Value,Texas,Southwest Airlines Co.,WN,19393.0
999998,Tennessee,Null_Value,Delta Air Lines Inc.,DL,19790.0


In [39]:
train['']

,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
0,4,15,NaN,NaN,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,None
1,8,15,740.0,1024.0,0,0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,None
2,9,6,1610.0,1805.0,0,0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,None
3,7,10,905.0,1735.0,0,0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,None
4,1,11,900.0,1019.0,0,0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,9,18,936.0,1243.0,0,0,ORD,13930,None,PHL,14100,None,678.0,United Air Lines Inc.,UA,19977.0,N477UA,None
999996,5,30,920.0,1028.0,0,0,FAR,11637,None,MSP,13487,Minnesota,223.0,SkyWest Airlines Inc.,DL,NaN,N439SW,None
999997,6,28,800.0,1340.0,0,0,OAK,13796,None,HOU,12191,Texas,1642.0,Southwest Airlines Co.,WN,19393.0,N230WN,None
999998,9,27,1613.0,1824.0,0,0,BNA,10693,Tennessee,ATL,10397,None,214.0,Delta Air Lines Inc.,DL,19790.0,N968DL,None


In [1]:
# from pycaret.classification import *

# test_1 = setup(data=train, target='Delay', train_size=.75, fold=5)

from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd

train = pd.read_parquet('train.parquet').drop('ID', axis=1)
test = pd.read_parquet('test.parquet').drop('ID', axis=1)

In [2]:
train = TabularDataset(train)
test = TabularDataset(test)

predictor = TabularPredictor(label='Delay', eval_metric='log_loss').fit(train)

No path specified. Models will be saved in: "AutogluonModels\ag-20230425_044954\"
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230425_044954\"
AutoGluon Version:  0.7.0
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Train Data Rows:    1000000
Train Data Columns: 17
Label Column: Delay
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  ['Not_Delayed', 'Delayed']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])


ValueError: Label column cannot contain non-finite values (NaN, Inf, Ninf). First invalid label at idx: 0